In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import time
import xgboost as xgb

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [3]:
# Load data
train = pl.read_parquet('data/train.parquet').drop('__index_level_0__')

# 刪除空字串

In [4]:
# 確認字串欄位
str_cols = [c for c in train.columns if train[c].dtype == pl.Utf8 or train[c].dtype == pl.String]

# 確認數值欄位
numeric_cols = [c for c in train.columns if train[c].dtype.is_numeric()]

# 將空字串視為 null，並填 missing
string_exprs = [
    pl.when(pl.col(c).str.strip_chars() == "")
      .then(None)
      .otherwise(pl.col(c))
      .alias(c)
    for c in str_cols
]

# 先把空字串改成 null
train_replaced = train.with_columns(string_exprs)

# 再一次性做 fill_null
train_filled = train_replaced.with_columns(
    [pl.col(c).fill_null("missing") for c in str_cols] +
    [pl.col(c).fill_null(0) for c in numeric_cols]
)

print("✅ 所有空字串與null已處理完畢")


✅ 所有空字串與null已處理完畢


# 處理duration

In [5]:
duration_cols = [
    "legs0_duration",
    "legs1_duration",
    "legs0_segments0_duration",
    "legs0_segments1_duration",
    "legs0_segments2_duration",
    "legs0_segments3_duration",
    "legs1_segments0_duration",
    "legs1_segments1_duration",
    "legs1_segments2_duration",
    "legs1_segments3_duration"
]
duration_exprs = [
    pl.when(pl.col(c).is_in([None, "missing"]))
      .then(0)
      .otherwise(
          pl.col(c).str.extract(r"^(\d+):", 1).cast(pl.Int64) * 60 +
          pl.col(c).str.extract(r":(\d+):", 1).cast(pl.Int64)
      )
      .alias(c)
    for c in duration_cols if c in train_filled.columns
]

train_filled = train_filled.with_columns(duration_exprs)


# add time

In [6]:
time_exprs = []
for col in ("legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"):
    if col in train_filled.columns:
        # 把 "missing" 統一變 null
        cleaned_col = (
            pl.when(pl.col(col) == "missing")
              .then(None)
              .otherwise(pl.col(col))
        )

        # 轉 datetime
        dt = cleaned_col.str.to_datetime(strict=False)

        # 小時
        h = dt.dt.hour()

        # 四段時段
        period = (
            pl.when(h.is_between(0,5))
              .then(0)
              .when(h.is_between(6,11))
              .then(1)
              .when(h.is_between(12,17))
              .then(2)
              .when(h.is_between(18,23))
              .then(3)
        )

        time_exprs.extend([
            h.fill_null(-1).alias(f"{col}_hour"),
            dt.dt.weekday().fill_null(-1).alias(f"{col}_weekday"),
            (
                ((h >= 6) & (h <= 9)) | ((h >= 17) & (h <= 20))
            ).cast(pl.Int32).fill_null(-1).alias(f"{col}_business_time"),
            period.fill_null(-1).alias(f"{col}_day_period")
        ])

train_filled = train_filled.with_columns(time_exprs)

In [7]:
# 時間欄位
time_cols = ["legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"]

# 來回標記
round_trip_flag = (
    (
        (
            pl.col("legs1_departureAt").is_not_null() & (pl.col("legs1_departureAt") != "missing")
        )
        |
        (
            pl.col("legs1_arrivalAt").is_not_null() & (pl.col("legs1_arrivalAt") != "missing")
        )
    )
    .cast(pl.Int8)
    .alias("is_round_trip")
)



# 加入DataFrame
train_filled = train_filled.with_columns([round_trip_flag])


In [8]:
# 把 legs0_departureAt 轉成 datetime
depart_dt = pl.col("legs0_departureAt").str.to_datetime(strict=False)

# requestDate 已經是 datetime
request_dt = pl.col("requestDate")

# 計算相減（Duration）
duration_ms = (depart_dt - request_dt).dt.total_milliseconds()

# 轉成天數
days_diff = (
    (duration_ms / (1000 * 60 * 60 * 24))
    .floor()
    .cast(pl.Int32)
    .fill_null(-1)
    .alias("days_before_departure")
)

# 加到 DataFrame
train_filled = train_filled.with_columns([days_diff])


# convert bool

In [9]:
# 先找出所有布林欄位
bool_cols = [c for c in train_filled.columns if train_filled[c].dtype == pl.Boolean]

print("✅ Boolean 欄位：")
print(bool_cols)

# 把所有布林欄位轉成 0/1
train_filled = train_filled.with_columns([
    pl.col(c).cast(pl.Int8) for c in bool_cols
])


✅ Boolean 欄位：
['bySelf', 'isAccess3D', 'isVip', 'sex']


# 加上是否直飛跟轉機幾次

In [10]:
# Legs0 segments1~3 有多少存在
legs0_segment_cols = [
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments3_departureFrom_airport_iata"
]

legs1_segment_cols = [
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments3_departureFrom_airport_iata"
]

# legs0轉機次數
legs0_num_segments = (
    pl.sum_horizontal([
        ((pl.col(c).is_not_null()) & (pl.col(c) != "missing")).cast(pl.Int8)
        for c in legs0_segment_cols
    ])
    .alias("legs0_num_transfers")
)

# legs1轉機次數
legs1_num_segments = (
    pl.sum_horizontal([
        ((pl.col(c).is_not_null()) & (pl.col(c) != "missing")).cast(pl.Int8)
        for c in legs1_segment_cols
    ])
    .alias("legs1_num_transfers")
)

# 寫入轉機次數
train_filled = train_filled.with_columns([
    legs0_num_segments,
    legs1_num_segments
])

# 是否直飛
train_filled = train_filled.with_columns([
    (pl.col("legs0_num_transfers") == 0).cast(pl.Int8).alias("legs0_is_direct"),
    (pl.col("legs1_num_transfers") == 0).cast(pl.Int8).alias("legs1_is_direct")
])

# 全程直飛
train_filled = train_filled.with_columns([
    (
        (pl.col("legs0_is_direct") == 1) & (pl.col("legs1_is_direct") == 1)
    ).cast(pl.Int8).alias("both_legs_direct")
])


In [ ]:
import polars as pl
import pickle
import gc

# 假設 train_filled 已經存在

# 所有欄位
all_cols = train_filled.columns

label_enc_cols = []

# Aircraft code
aircraft_cols = [c for c in all_cols if c.endswith("_aircraft_code")]
label_enc_cols.extend(aircraft_cols)

# Flight Number
flightnum_cols = [c for c in all_cols if c.endswith("_flightNumber")]
label_enc_cols.extend(flightnum_cols)

# Airport / City
airport_cols = [c for c in all_cols if "_arrivalTo_airport_" in c or "_departureFrom_airport_" in c]
label_enc_cols.extend(airport_cols)

# Carrier
carrier_cols = [c for c in all_cols if c.endswith("_marketingCarrier_code") or c.endswith("_operatingCarrier_code")]
label_enc_cols.extend(carrier_cols)

# searchRoute
label_enc_cols.append("searchRoute")

# frequentFlyer
label_enc_cols.append("frequentFlyer")

# ✅先做frequentFlyer特徵，只取需要的欄
frequentFlyer_features = (
    train_filled.select([
        pl.col("frequentFlyer").cast(pl.Utf8).fill_null("missing").alias("frequentFlyer")
    ])
    .with_columns([
        (
            (pl.col("frequentFlyer") != "")
            & (pl.col("frequentFlyer") != "missing")
        ).cast(pl.Int8).alias("has_frequentFlyer"),
        (
            pl.col("frequentFlyer").map_elements(
                lambda s: 0 if s in ("", "missing") else s.count("/") + 1,
                return_dtype=pl.Int32
            ).alias("n_ff_programs")
        )
    ])
)

# 先把frequentFlyer衍生欄拿出
ff_has_series = frequentFlyer_features["has_frequentFlyer"]
ff_count_series = frequentFlyer_features["n_ff_programs"]

# ✅做Label Encoding
label_encoding_exprs = [
    (pl.col(c).rank("dense") - 1).fill_null(-1).cast(pl.Int32).alias(c)
    for c in label_enc_cols
]

label_encoders = {}
for c in label_enc_cols:
    mapping_df = (
        train_filled.select(pl.col(c))
        .unique()
        .with_columns(
            (pl.col(c).rank("dense") - 1).fill_null(-1).cast(pl.Int32).alias("rank_id")
        )
        .sort("rank_id")
    )
    label_encoders[c] = mapping_df.to_dict(as_series=False)

aircraft_missing_exprs = [
    pl.col(c).is_null().cast(pl.Int8).alias(c + "_is_missing")
    for c in aircraft_cols
]

# ✅先做Label Encoding + Aircraft缺失
train_filled = train_filled.with_columns(
    label_encoding_exprs + aircraft_missing_exprs
)

# ✅最後把ff_*加回去
train_filled = train_filled.with_columns([
    ff_has_series.alias("has_frequentFlyer"),
    ff_count_series.alias("n_ff_programs")
])
del ff_has_series, ff_count_series
gc.collect()

print("✅ 完成所有Label Encoding 與特徵工程！")

transform_config = {
    "label_encoders": label_encoders,
    "aircraft_cols": aircraft_cols,
    "frequentFlyer_derived": ["has_frequentFlyer", "n_ff_programs"]
}

with open("transform_config_rank.pkl", "wb") as f:
    pickle.dump(transform_config, f)

print("✅ 已儲存 transform_config_rank.pkl")


✅ 完成所有Label Encoding 與特徵工程！
✅ 已儲存 transform_config_rank.pkl


In [ ]:
del label_encoding_exprs, aircraft_missing_exprs, train, train_replaced, frequentFlyer_features
gc.collect()

31

In [18]:
# legs0所有marketingCarrier
legs0_marketing_cols = [
    f"legs0_segments{s}_marketingCarrier_code"
    for s in range(4) if f"legs0_segments{s}_marketingCarrier_code" in train_filled.columns
]
legs1_marketing_cols = [
    f"legs1_segments{s}_marketingCarrier_code"
    for s in range(4) if f"legs1_segments{s}_marketingCarrier_code" in train_filled.columns
]

# legs0 主carrier (取第一個不為null)
legs0_main_carrier = (
    pl.coalesce([pl.col(c) for c in legs0_marketing_cols])
    .alias("legs0_main_carrier")
)

# legs1 主carrier
legs1_main_carrier = (
    pl.coalesce([pl.col(c) for c in legs1_marketing_cols])
    .alias("legs1_main_carrier")
)

train_filled = train_filled.with_columns([
    legs0_main_carrier,
    legs1_main_carrier
])

# legs0是否一致
legs0_all_same = (
    pl.when(pl.col("legs0_num_transfers") == 0)
    .then(1)
    .otherwise(
        pl.all_horizontal([
            (pl.col(c) == pl.col("legs0_main_carrier")) & pl.col(c).is_not_null()
            for c in legs0_marketing_cols
        ]).cast(pl.Int8)
    )
    .alias("legs0_all_segments_carrier_same")
)

# legs1是否一致
legs1_all_same = (
    pl.when(pl.col("legs1_num_transfers") == 0)
    .then(1)
    .otherwise(
        pl.all_horizontal([
            (pl.col(c) == pl.col("legs1_main_carrier")) & pl.col(c).is_not_null()
            for c in legs1_marketing_cols
        ]).cast(pl.Int8)
    )
    .alias("legs1_all_segments_carrier_same")
)

# 加入腿一致性
train_filled = train_filled.with_columns([
    legs0_all_same,
    legs1_all_same
])

# 最後判斷兩腿是否都是同一家
both_legs_all_same = (
    (
        (pl.col("legs0_all_segments_carrier_same") == 1) &
        (pl.col("legs1_all_segments_carrier_same") == 1) &
        (pl.col("legs0_main_carrier") == pl.col("legs1_main_carrier")) &
        pl.col("legs0_main_carrier").is_not_null() &
        pl.col("legs1_main_carrier").is_not_null()
    ).cast(pl.Int8)
    .alias("both_legs_carrier_all_same")
)

train_filled = train_filled.with_columns([
    both_legs_all_same
])

print("✅ 完成 legs0/legs1 主Carrier一致判斷！")


✅ 完成 legs0/legs1 主Carrier一致判斷！


In [21]:
train_filled

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs0_departureAt_weekday,legs0_departureAt_business_time,legs0_departureAt_day_period,legs0_arrivalAt_hour,legs0_arrivalAt_weekday,legs0_arrivalAt_business_time,legs0_arrivalAt_day_period,legs1_departureAt_hour,legs1_departureAt_weekday,legs1_departureAt_business_time,legs1_departureAt_day_period,legs1_arrivalAt_hour,legs1_arrivalAt_weekday,legs1_arrivalAt_business_time,legs1_arrivalAt_day_period,is_round_trip,days_before_departure,legs0_num_transfers,legs1_num_transfers,legs0_is_direct,legs1_is_direct,both_legs_direct,legs0_segments0_aircraft_code_is_missing,legs0_segments1_aircraft_code_is_missing,legs0_segments2_aircraft_code_is_missing,legs0_segments3_aircraft_code_is_missing,legs1_segments0_aircraft_code_is_missing,legs1_segments1_aircraft_code_is_missing,legs1_segments2_aircraft_code_is_missing,legs1_segments3_aircraft_code_is_missing,has_frequentFlyer,n_ff_programs,legs0_main_carrier,legs1_main_carrier,legs0_all_segments_carrier_same,legs1_all_segments_carrier_same,both_legs_carrier_all_same
i64,i8,i64,i64,i32,i64,i8,i8,str,str,i64,i32,i32,i32,f64,f64,f64,i32,i64,i32,i32,i32,f64,i32,i32,i32,f64,f64,f64,i32,i64,i32,i32,i32,f64,i32,i32,…,i8,i32,i32,i8,i8,i32,i32,i8,i8,i32,i32,i8,i8,i32,i32,i8,i32,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i32,i32,i32,i8,i8,i8
0,1,57323,0,95,36,0,0,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""",160,105,217,238,1.0,0.0,1.0,319,160,1226,88,117,9.0,102,424,475,0.0,0.0,0.0,453,0,6576,149,200,0.0,81,242,…,6,0,2,16,6,0,2,9,2,1,1,14,2,0,2,1,29,0,0,1,1,1,0,0,0,0,0,0,0,0,1,3,88,77,1,1,0
1,1,57323,123,95,36,1,0,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,89,335,360,1.0,0.0,1.0,319,170,3841,122,160,4.0,86,188,215,1.0,0.0,1.0,303,80,3295,114,153,4.0,81,242,…,6,1,1,14,6,0,2,22,2,0,3,8,3,1,1,1,29,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,122,109,0,0,0
2,1,57323,0,95,36,0,0,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,89,335,360,1.0,0.0,1.0,319,170,3841,122,160,4.0,86,188,215,1.0,0.0,1.0,303,80,3295,114,153,4.0,81,242,…,6,1,1,14,6,0,2,22,2,0,3,8,3,1,1,1,29,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,122,109,0,0,0
3,1,57323,123,95,36,1,0,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,89,335,360,1.0,0.0,1.0,319,170,3841,122,160,4.0,86,188,215,1.0,0.0,1.0,303,80,3295,114,153,4.0,81,242,…,6,1,1,14,6,0,2,22,2,0,3,8,3,1,1,1,29,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,122,109,0,0,0
4,1,57323,0,95,36,0,0,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,89,335,360,1.0,0.0,1.0,319,170,3841,122,160,4.0,86,188,215,1.0,0.0,1.0,303,80,3295,114,153,4.0,81,242,…,6,1,1,14,6,0,2,22,2,0,3,8,3,1,1,1,29,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,122,109,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
18146427,1,54154,44,371,36,1,0,"""2024-11-06T09:45:00""","""2024-11-05T20:50:00""",655,39,379,405,0.0,0.0,1.0,349,200,1530,141,184,3.0,73,340,373,0.0,0.0,1.0,

# remove time

In [19]:
# 先找出所有布林欄位
string_cols = [c for c in train_filled.columns if train_filled[c].dtype == pl.String]

print("✅ String 欄位：")
print(string_cols)


✅ String 欄位：
['legs0_arrivalAt', 'legs0_departureAt', 'legs1_arrivalAt', 'legs1_departureAt', 'ranker_id']


In [20]:
# 儲存成 Parquet 格式
train_filled.write_parquet("data/train_filled.parquet")

print("✅ 已將 train_filled 儲存為 data/train_filled.parquet")


✅ 已將 train_filled 儲存為 data/train_filled.parquet


In [ ]:
# 先讀 parquet
test = pl.read_parquet('data/test_filled.parquet')
train = pl.read_parquet('data/train_filled.parquet')  # 這裡你可能打錯了，train 路徑

# 取欄位名稱
train_cols = set(train.columns)
test_cols = set(test.columns)

# 列印比較
print("✅ Train columns:", len(train_cols))
print("✅ Test columns:", len(test_cols))

print("\n🎯 Train 中有但 Test 沒有的欄位:")
print(sorted(train_cols - test_cols))

print("\n🎯 Test 中有但 Train 沒有的欄位:")
print(sorted(test_cols - train_cols))

# 如果完全一樣
if train_cols == test_cols:
    print("\n✅ Train 和 Test 欄位完全一致！")
